In [ ]:
!pip install langchain
!pip install chromadb
!pip install tiktoken
!pip install langchain_community

import tiktoken

from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
import pandas as pd

!pip install -q openai
!pip install -q python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 6.0 MB/s eta 0:00:00


In [ ]:
df=pd.read_csv("/content/drive/MyDrive/질환증상크롤링.csv")

In [ ]:
df=df.fillna('-')

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=0)
all_splits = []
for text in df:
    splits = text_splitter.split_text(text)
    all_splits.extend(splits)

documents_splits = text_splitter.create_documents(all_splits)


In [ ]:
vectorstore=Chroma.from_documents(documents=documents_splits,
                                  embedding=OpenAIEmbeddings(openai_api_key=""))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [ ]:
question='기억력이 약간 저하되는데 무슨 질환이지?'

docs=vectorstore.similarity_search(question)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

retriever=vectorstore.as_retriever(search_kwargs={"k":1})
llm=ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0,openai_api_key="")
chain=RetrievalQA.from_chain_type(retriever=retriever,
                                                  llm=llm,
                                                  chain_type="stuff",
                                                  return_source_documents=True,
                                                  )


result=chain(question)
result

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'query': '기억력이 약간 저하되는데 무슨 질환이지?',
 'result': '기억력이 약간 저하되는 증상은 다양한 원인이 있을 수 있습니다. 이는 스트레스, 수면 부족, 우울증, 뇌 손상, 뇌졸중, 알츠하이머병 등 다양한 요인에 의해 발생할 수 있습니다. 정확한 진단을 받기 위해서는 의사와 상담하여 진단을 받는 것이 중요합니다.',
 'source_documents': [Document(page_content='질환명')]}

In [ ]:
from langchain.prompts.chat import (ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate)
system_template="""다음 내용을 참조하여 사용자의 질문에 간단히 답변줄 것.
문서와 질무에 대한 요약(summaries)이 주어지면 참조("SOURCES")를 사용하여 최종 답변을 작성할 것.
답을 모르면 "모른다"고 말하고 답을 만들어 내려고 하지 말것.
--------------------------------------------------------------------------------------------------
{summaries}

한국어와 마크다운 형식으로 답변할 것:"""

message=[SystemMessagePromptTemplate.from_template(system_template),
         HumanMessagePromptTemplate.from_template("{question}")]
prompt=ChatPromptTemplate.from_messages(message)
prompt.input_variables = ["question", "document_variable_name"]


In [ ]:
from langchain.chains import RetrievalQA
chain_type_kwargs={}
chain=RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever, chain_type_kwargs=chain_type_kwargs,
                                  return_source_documents=True,)

In [ ]:
questin='편두통이 심한 거 같아. 속도 메스껍고. 무슨 질환이지?'
result=chain(questin)
result

{'query': '편두통이 심한 거 같아. 속도 메스껍고. 무슨 질환이지?',
 'result': '편두통은 다양한 원인에 의해 발생할 수 있는 증상이며, 스트레스, 불규칙한 수면, 불규칙한 식습관, 혈액순환의 문제, 혈압이나 호르몬의 변화 등이 원인이 될 수 있습니다. 하지만 정확한 진단을 받기 위해서는 의사를 방문하여 상세한 검사를 받는 것이 좋습니다. 특히 편두통이 심하다면 의사의 진료를 받아보시는 것이 좋습니다.',
 'source_documents': [Document(page_content='증상')]}